# LOESS

## Enfoque General

**1. Preprocesamiento:**

Se extraen las columnas de provisión (ya transformadas a porcentaje multiplicándolas por 100) y se filtran aquellos meses en los que se disponga de datos (por ejemplo, donde el valor es mayor a cero).

**2. Ajuste de la Curva Suave (LOESS):**

Para cada cliente se usa la función lowess del paquete statsmodels para ajustar una curva suave a los datos. Esto permite capturar relaciones complejas sin asumir una forma funcional predefinida.

**3. Estimación de la Tendencia:**

Una vez que se obtiene la curva suavizada, se calcula la derivada aproximada (usando, por ejemplo, la función np.gradient) con respecto al tiempo. La derivada en cada punto indica la tasa de cambio local de la provisión. Se puede resumir la tendencia tomando la derivada promedio (o evaluando la derivada en puntos clave, como el medio del intervalo) para decidir si, en promedio, la provisión aumenta o disminuye a lo largo del tiempo.

**4. Interpretación:**

* **Derivada promedio positiva:** Indica que, en promedio, el porcentaje de provisión está aumentando con el tiempo.
* **Derivada promedio negativa:** Indica que, en promedio, está disminuyendo.
* **Derivada promedio cercana a cero:** Sugiere que no hay cambios importantes en la tendencia, o que la variación es muy irregular.

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

def calcular_tendencia_no_lineal(df):
    """
    Calcula la tendencia no lineal en el porcentaje de provisión (provision/saldo * 100)
    para cada cliente usando un ajuste LOESS y estima la derivada promedio de la curva suavizada.
    
    Se añaden las siguientes columnas al DataFrame:
      - tendencia_no_lineal: Derivada promedio de la curva suavizada (análogo a la pendiente en un modelo no lineal).
      - meses_con_datos: Número de meses con datos válidos (provisión > 0).
    
    Parámetros
    ----------
    df : pd.DataFrame
        DataFrame que contiene 'cliente_id' y columnas 'provision_mes1' a 'provision_mes12'.
        Se asume que cada valor en provision_mesX ya es el cociente provision/saldo (sin multiplicar por 100).
    
    Retorna
    -------
    df : pd.DataFrame
        El DataFrame original con las nuevas columnas añadidas.
    """
    
    # Lista de columnas de provisión
    meses_cols = [f'provision_mes{i}' for i in range(1, 13)]
    
    # Inicialización de las nuevas columnas
    df["tendencia_no_lineal"] = np.nan
    df["meses_con_datos"] = np.nan
    
    # Iterar sobre cada cliente
    for idx, row in df.iterrows():
        # Extraer y convertir a float los valores, multiplicándolos por 100 para expresarlos en porcentaje.
        # Esto transforma el cociente (provision/saldo) a un porcentaje.
        provisiones = np.array([float(row[col]) * 100 for col in meses_cols])
        
        # Identificar los índices (meses) en que la provisión es mayor a cero.
        idx_validos = np.where(provisiones > 0)[0]
        # Renumerar los meses válidos para que empiecen en 1
        if len(idx_validos) > 0:
            meses_validos = (idx_validos + 1).astype(float)
        else:
            meses_validos = np.array([])
        
        # Filtrar los valores de provisión correspondientes
        provisiones_validas = provisiones[idx_validos]
        
        # Guardar el número de meses con datos
        df.at[idx, "meses_con_datos"] = len(meses_validos)
        
        if len(meses_validos) > 1:
            # Ajuste LOESS para suavizar la relación no lineal
            # El parámetro frac determina la fracción de datos usados en cada estimación local
            frac = 0.6  # Este parámetro se puede ajustar según la variabilidad de los datos
            loess_result = sm.nonparametric.lowess(provisiones_validas, meses_validos, frac=frac, return_sorted=True)
            x_suave = loess_result[:, 0]
            y_suave = loess_result[:, 1]
            
            # Calcular la derivada aproximada de la curva suavizada
            derivada = np.gradient(y_suave, x_suave)
            
            # Se toma como tendencia la derivada promedio
            tendencia_promedio = np.mean(derivada)
        else:
            tendencia_promedio = np.nan
        
        df.at[idx, "tendencia_no_lineal"] = tendencia_promedio
    
    return df

# Ejemplo de uso:
# Cargar el DataFrame (se asume que ya contiene cliente_id y provision_mes1 ... provision_mes12)
df = pd.read_csv("datos_provision.csv")

# Calcular la tendencia no lineal y actualizar el DataFrame
df_actualizado = calcular_tendencia_no_lineal(df)

# Guardar o visualizar el DataFrame actualizado
df_actualizado.to_csv("datos_provision_actualizados_no_lineal.csv", index=False)
print(df_actualizado.head())


## Explicación y Consideraciones

**1. Transformación a Porcentaje:**

Se multiplica cada valor extraído de las columnas por 100. Esto convierte la razón provisión/saldo a un porcentaje, facilitando su interpretación.

**2. Filtrado de Datos Válidos:**

Solo se consideran los meses en los que el valor de provisión es mayor a cero, ya que meses en cero pueden representar la ausencia de información útil.


**3. Uso de LOESS:**

La función lowess de statsmodels ajusta una curva suave a los datos.
El parámetro frac controla la cantidad de puntos utilizados en cada estimación local y puede ajustarse según la densidad y variabilidad de los datos.

**4. Estimación de la Derivada:**

Se utiliza np.gradient para aproximar la derivada de la curva suavizada. La derivada en cada punto indica la tasa de cambio local del porcentaje de provisión. Se promedia esta derivada para obtener una medida global de la tendencia.

**5. Interpretación de la Tendencia No Lineal:**

* Tendencia positiva (derivada promedio > 0): El porcentaje de provisión, en promedio, está aumentando a lo largo del tiempo.

* Tendencia negativa (derivada promedio < 0): Está disminuyendo.

* Tendencia cercana a cero: No hay cambios significativos o la variación es muy irregular.


Si los datos muestran una tendencia no lineal, el uso de técnicas de suavizado como LOESS es una alternativa robusta a la regresión lineal. Esta metodología permite capturar patrones complejos y estimar una “pendiente” o tasa de cambio promedio a partir de la derivada de la curva suavizada, ofreciendo así una visión más precisa de la evolución del porcentaje de provisión en relación con el saldo del cliente.

# Ajuste Polinómico con Selección Automática del Grado Óptimo

Otra alternativa para modelar la tendencia en el porcentaje de provisión (calculado como *provisión/saldo* y multiplicado por 100) mediante regresión polinómica. La idea es ajustar, para cada cliente, varios modelos polinómicos de diferentes grados y seleccionar el grado óptimo utilizando un criterio de selección (por ejemplo, AIC). Esto permite que el modelo se adapte a la complejidad de los datos, evitando imponer un modelo de grado fijo.



## Metodología Paso a Paso

### 1. Transformación de Datos

- **Extracción y Conversión:**
  - Para cada cliente se extraen los valores de provisión de los 12 meses.
  - Cada valor se multiplica por 100 para convertir la relación *provisión/saldo* a porcentaje.
  
- **Filtrado:**
  - Se filtran aquellos meses en los que la provisión es mayor a cero (considerados datos válidos).

### 2. Selección del Grado del Polinomio

- **Ajuste de Modelos:**
  - Para los puntos \((x, y)\) (donde \(x\) es el número del mes y \(y\) es el porcentaje de prvision), se ajusta un polinomio de grado para diferentes grados usando mínimos cuadrados.
  
- **Criterio AIC:**
  - Se calcula el Residual Sum of Squares (RSS) para cada ajuste.
  - Se utiliza la siguiente fórmula para el AIC:
  
    $$
    \text{AIC} = n \cdot \ln\left(\frac{\text{RSS}}{n}\right) + 2k,
    $$
    donde:
    - \( n \) es el número de puntos (meses válidos).
    - \( k \) es el número de parámetros del modelo (grado \(d\) + 1).
  - Se recorre el rango de grados candidatos (por ejemplo, de 1 hasta el mínimo entre un grado máximo predefinido y \(n-1\)) y se selecciona el grado que minimice el AIC.

### 3. Ajuste del Modelo Seleccionado y Cálculo de la Tendencia

- **Reajuste del Polinomio Óptimo:**
  - Con el grado óptimo seleccionado, se vuelve a ajustar el polinomio a los datos.
  
- **Derivada Analítica:**
  - Se calcula la derivada del polinomio de forma analítica.
    - Por ejemplo, si el polinomio es:
    
      $$
      f(x) = a_d x^d + a_{d-1} x^{d-1} + \cdots + a_1 x + a_0,
      $$
      su derivada es:

      $$
      f'(x) = d\,a_d x^{d-1} + (d-1)\,a_{d-1} x^{d-2} + \cdots + a_1.
      $$
  
- **Evaluación en el Punto Medio:**
  - Se evalúa la derivada en el punto medio de los meses válidos (por ejemplo, el punto medio de los meses con datos validos) para obtener una medida de la tasa de cambio o "tendencia".
    - Una derivada positiva indica que el porcentaje está aumentando.
    - Una derivada negativa indica que el porcentaje está disminuyendo.

### 4. Almacenamiento de Resultados

- Para cada cliente se almacena:
  - La **tendencia polinómica** (la derivada evaluada en el punto medio).
  - El **grado óptimo** seleccionado.
  - El número de **meses con datos válidos**.

---



In [ ]:
import pandas as pd
import numpy as np

def calcular_tendencia_polinomica_variable(df, max_degree=5, epsilon=1e-8):
    """
    Calcula la tendencia en el porcentaje de provisión (provision/saldo * 100)
    para cada cliente usando regresión polinómica con selección automática del grado óptimo.

    Para cada cliente se realiza lo siguiente:
      1. Se extraen los datos de provisión para cada mes, se multiplican por 100 y se filtran
         aquellos meses en que la provisión es mayor a cero.
      2. Se ajusta un polinomio a los datos para diferentes grados (desde 1 hasta el mínimo entre max_degree
         y (número de datos - 1)).
      3. Se calcula el AIC de cada modelo:
             AIC = n * ln(RSS/n) + 2 * (grado + 1)
         y se selecciona el grado que minimice el AIC.
      4. Se vuelve a ajustar el polinomio óptimo y se calcula su derivada.
      5. Se evalúa la derivada en el punto medio de los meses válidos como medida de la tendencia.
    
    Parámetros:
    -----------
    df : pd.DataFrame
        DataFrame que contiene 'cliente_id' y las columnas 'provision_mes1' a 'provision_mes12'.
        Se asume que cada valor es la relación (provision/saldo) sin multiplicar por 100.
    max_degree : int, opcional (default=5)
        Grado máximo que se considerará para el polinomio.
    epsilon : float, opcional (default=1e-8)
        Valor pequeño para evitar problemas con log(0) en el cálculo del AIC.
    
    Retorna:
    --------
    df : pd.DataFrame
        El DataFrame original con las nuevas columnas:
          - 'tendencia_polinomica_var': derivada evaluada en el punto medio (medida de la tendencia).
          - 'grado_optimo': grado del polinomio seleccionado.
          - 'meses_con_datos': número de meses con datos válidos.
    """
    # Lista de columnas de provisión
    meses_cols = [f'provision_mes{i}' for i in range(1, 13)]
    
    # Inicializar las columnas de salida
    df["tendencia_polinomica_var"] = np.nan
    df["grado_optimo"] = np.nan
    df["meses_con_datos"] = np.nan

    for idx, row in df.iterrows():
        # Extraer valores y convertir a float, multiplicándolos por 100 para expresar en porcentaje.
        provisiones = np.array([float(row[col]) * 100 for col in meses_cols])
        
        # Filtrar los meses donde la provisión es mayor a cero.
        idx_validos = np.where(provisiones > 0)[0]
        meses_validos = (idx_validos + 1).astype(float)  # Meses numerados: 1, 2, ..., n
        provisiones_validas = provisiones[idx_validos]
        
        n_points = len(meses_validos)
        df.at[idx, "meses_con_datos"] = n_points
        
        # Si no hay suficientes datos, no se puede ajustar ningún modelo.
        if n_points < 2:
            df.at[idx, "tendencia_polinomica_var"] = np.nan
            df.at[idx, "grado_optimo"] = np.nan
            continue
        
        # Definir el rango de grados a considerar: desde 1 hasta min(max_degree, n_points - 1)
        grado_maximo = min(max_degree, n_points - 1)
        
        best_AIC = np.inf
        best_degree = None
        best_coeffs = None
        
        # Loop sobre grados candidatos
        for degree in range(1, grado_maximo + 1):
            try:
                # Ajustar el polinomio de grado 'degree'
                coeffs = np.polyfit(meses_validos, provisiones_validas, deg=degree)
                # Predecir los valores
                y_pred = np.polyval(coeffs, meses_validos)
                # Calcular el Residual Sum of Squares (RSS)
                RSS = np.sum((provisiones_validas - y_pred) ** 2)
                # Evitar log(0)
                RSS = max(RSS, epsilon)
                # Número de parámetros (grado + 1)
                k = degree + 1
                # Calcular AIC: n * ln(RSS/n) + 2k
                AIC = n_points * np.log(RSS / n_points) + 2 * k
            except Exception as e:
                # En caso de fallo en el ajuste, saltar a la siguiente iteración.
                continue

            # Seleccionar el modelo con AIC mínimo
            if AIC < best_AIC:
                best_AIC = AIC
                best_degree = degree
                best_coeffs = coeffs

        # Si se encontró un modelo adecuado, se utiliza para calcular la tendencia.
        if best_coeffs is not None:
            # Calcular la derivada del polinomio ajustado
            d_coeffs = np.polyder(best_coeffs)
            # Evaluar en el punto medio de los meses válidos
            x_mid = (meses_validos.min() + meses_validos.max()) / 2.0
            tendencia = np.polyval(d_coeffs, x_mid)
        else:
            tendencia = np.nan
            best_degree = np.nan
        
        df.at[idx, "tendencia_polinomica_var"] = tendencia
        df.at[idx, "grado_optimo"] = best_degree

    return df

# # Ejemplo de uso:
# df = pd.read_csv("datos_provision.csv")  
# df_actualizado_poly = calcular_tendencia_polinomica_variable(df, max_degree=5)
# df_actualizado_poly.to_csv("datos_provision_actualizados_polinomica_variable.csv", index=False)
# print(df_actualizado_poly.head())

# Modelo Spline Suavizante para Ajustar la Tendencia en Datos No Lineales

Una tercera alternativa para modelar la tendencia en el porcentaje de provisión (calculado como *provisión/saldo* multiplicado por 100) puede ser mediante un **modelo spline suavizante**. El método spline permite ajustar una curva flexible a los datos sin asumir una forma funcional global, capturando la no linealidad de forma continua.

---

## Metodología Paso a Paso

### 1. Transformación de Datos

- **Extracción y Conversión:**
  - Se extraen los valores de provisión de los 12 meses para cada cliente.
  - Cada valor se multiplica por 100 para convertir la relación *provisión/saldo* a porcentaje.
  
- **Filtrado:**
  - Se filtran aquellos meses en los que la provisión es mayor a cero, considerándolos como datos válidos para el ajuste.

### 2. Ajuste del Modelo Spline

- **Selección de Datos Válidos:**
  - Se utiliza el conjunto de puntos \((x, y)\), donde \(x\) es el número del mes (por ejemplo, 1, 2, 3, …) y \(y\) es el porcentaje de provisión.

- **Ajuste del Spline Suavizante:**
  - Se utiliza un spline suavizante mediante la función `UnivariateSpline` de `scipy.interpolate`.
  - El spline se ajusta a los datos válidos. Se puede especificar un parámetro de suavizado, `s_factor`, que controla el equilibrio entre la fidelidad a los datos y la suavidad de la curva.

### 3. Cálculo de la Tendencia

- **Derivada del Spline:**
  - Una vez ajustado el spline, se calcula su derivada. Esta derivada representa la tasa de cambio local del porcentaje de provisión.

- **Evaluación en un Punto Representativo:**
  - Se evalúa la derivada en un punto representativo, por ejemplo, el punto medio de los meses válidos:
    $$
    x_{\text{mid}} = \frac{\min(x) + \max(x)}{2}
    $$
  - El valor resultante se interpreta como una medida de la tendencia:
    - Una derivada positiva indica que el porcentaje está aumentando.
    - Una derivada negativa indica que el porcentaje está disminuyendo.

### 4. Almacenamiento de Resultados

- Para cada cliente se almacena:
  - La **tendencia spline** (la derivada evaluada en el punto medio).
  - El número de **meses con datos válidos**.

---

In [ ]:
import pandas as pd
import numpy as np
from scipy.interpolate import UnivariateSpline

def calcular_tendencia_spline(df, s_factor=1.0):
    """
    Calcula la tendencia no lineal en el porcentaje de provisión (provision/saldo * 100)
    para cada cliente usando un spline suavizante (UnivariateSpline).

    Procedimiento:
      1. Se extraen los datos de provisión para cada mes y se convierten a porcentaje.
      2. Se filtran los meses con provisión mayor a cero.
      3. Se ajusta un spline suavizante a los datos válidos utilizando UnivariateSpline.
      4. Se calcula la derivada del spline y se evalúa en el punto medio de los meses válidos para obtener la tendencia.
    
    Parámetros:
    -----------
    df : pd.DataFrame
        DataFrame que contiene 'cliente_id' y las columnas 'provision_mes1' a 'provision_mes12'.
    s_factor : float, opcional (default=1.0)
        Factor de suavizado para el spline. Un valor mayor implica una curva más suave.
    
    Retorna:
    --------
    df : pd.DataFrame
        El DataFrame original con las nuevas columnas:
          - 'tendencia_spline': derivada del spline evaluada en el punto medio (medida de la tendencia).
          - 'meses_con_datos': número de meses con datos válidos.
    """
    # Lista de columnas de provisión
    meses_cols = [f'provision_mes{i}' for i in range(1, 13)]
    
    # Inicializar las columnas de salida
    df["tendencia_spline"] = np.nan
    df["meses_con_datos"] = np.nan

    for idx, row in df.iterrows():
        # Extraer y convertir a float, multiplicando por 100 para expresar en porcentaje.
        provisiones = np.array([float(row[col]) * 100 for col in meses_cols])
        
        # Filtrar los meses donde la provisión es mayor a cero.
        idx_validos = np.where(provisiones > 0)[0]
        meses_validos = (idx_validos + 1).astype(float)  # Numerar los meses: 1, 2, 3, ...
        provisiones_validas = provisiones[idx_validos]
        
        # Registrar el número de meses con datos
        df.at[idx, "meses_con_datos"] = len(meses_validos)
        
        # Se requiere un mínimo de puntos para ajustar el spline (por ejemplo, al menos 4)
        if len(meses_validos) > 3:
            # Ajustar el spline suavizante a los datos válidos.
            spline = UnivariateSpline(meses_validos, provisiones_validas, s=s_factor)
            # Calcular el punto medio de los meses válidos
            x_mid = (meses_validos.min() + meses_validos.max()) / 2.0
            # Evaluar la derivada del spline en el punto medio
            tendencia = spline.derivative()(x_mid)
        else:
            tendencia = np.nan
        
        df.at[idx, "tendencia_spline"] = tendencia

    return df

# # Ejemplo de uso:
# df = pd.read_csv("datos_provision.csv")  # Se asume que el archivo contiene 'cliente_id' y 'provision_mes1' a 'provision_mes12'
# df_actualizado_spline = calcular_tendencia_spline(df, s_factor=1.0)
# df_actualizado_spline.to_csv("datos_provision_actualizados_spline.csv", index=False)
# print(df_actualizado_spline.head())